# 2. Feature Engineering

The following notebook provides functions for retrieving the following features for modeling. This notebook mostly serves as a dump for functions explored but processed dataframes per station and state can be retrieved by running *src/preprocess_trafficdata.py*. 

Some of the features that were explored are the following:

* Historical traffic volume
    * determined by *traffic_volume_counted_after*
* Temporal features
    * Cyclical timestamps (expressed as sin & cos values)
        * Month of year
        * Day of month
        * Day of week
        * Hour of day
    * Part of day (0-5AM, 6-11AM, 12-5PM, 6-11PM)
    * Weekend vs. weekday
* Spatial features
    * longitude
    * latitude
    * fips state code
    * station_id
    * urban/rural from functional_classification_name
    * Additionally, neighboring stations may also be retrieved by sorting and retrieving the [distances between the longitude and latitude values](https://mypages.iit.edu/~maslanka/3Dcoordinates.pdf) of stations
* Traffic features - Can be disregarded if volume trends to be checked are for locations relative to the station (no need for lane/direction in this case)
    * lane of travel
    * direction of travel 

Table of contents:
- [2.1 Prerequisites](#1)
- [2.2 Load raw data](#2)
- [2.3 Modify missing and incorrect data points](#3)
- [2.4 Functions for extracting features](#4)
- [2.5 Guide for using pre-processed data as input (LSTM)](#5)

<a id="1"></a>

## 2.1 Prerequisites

Import required libraries. Before running the notebook, it is assumed that the user has already installed the required libraries contained in *requirements.txt* .

In [1]:
import os
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import numpy as np
import pandas as pd
import pylab
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from tqdm.notebook import tqdm

from math import sin, cos, pi
import warnings
warnings.filterwarnings('ignore')

# Modify to adjust figure sizes
pylab.rcParams['figure.figsize'] = (12, 4)

Since the notebooks and utility functions are under src folder of the repository, `os.chdir("..")` is used as an easy way to move back one directory to access the folders. This serves as a workaround for the initial exploration. 

In [2]:
os.chdir("..")

In [3]:
import importlib
from utils.config import filenames 
from utils import datautils, preprocess

# Easy way to reload utility functions with changes
importlib.reload(datautils)
importlib.reload(preprocess)

<module 'utils.preprocess' from 'C:\\Users\\combi\\Documents\\gitrepos\\us-traffic\\src\\utils\\preprocess.py'>

<a id="2"></a>

## 2.2 Load raw data

Modify the DATA_LOCATION directory as needed. This configuration sets it as src/data in the repository. This assumes that the required data is already located within the directory e.g.
```
us-traffic
│   ...    
│
└───src
|   |   datasetdownloader.py
│   │   ...
│   │
│   └───data  <- (DATA_LOCATION)
|       |    dot_traffic_2015.txt.gz
|       |    dot_traffic_stations_2015.txt.gz
│       |
|       └─── ...
|
...
```

Alternatively, the user can also download the files via the datadownloader.py script under src.

In [4]:
DATA_LOCATION = os.path.join(os.getcwd(), 'data')

There are two main files under the dataset:
- Traffic data - dot_traffic_2015.txt.gz
- Traffic stations data - dot_traffic_stations_2015.txt.gz

Insert FIPS data. Assumes that the location of the FIPS files (fips_code.csv and fips_latlong.csv) are located under *src/data* within the us-traffic repository.

In [5]:
FIPS_LOCATION = DATA_LOCATION

In [6]:
fips_df, fips_loc_df = datautils.load_other_datasets(FIPS_LOCATION,
                                                    filenames["FIPS_CODE"],
                                                    filenames["FIPS_LOC"])
fips_state_ref = datautils.create_fips_ref(fips_df)

Loading FIPS state codes reference from 'fips_code.csv' ...
Loading approximate FIPS coordinates reference from 'fips_latlong.csv' ...
Finished loading data.


<a id="3"></a>

## 2.3 Modify missing and incorrect data points

Based on EDA done in the previous notebook, it was seen that there were some data points with missing values (0 or null) and some values with an incorrect offset with regards to the tens place (e.g. ~900 longitude values instead of ~90).

If new data such as traffic data from 2016 onwards will be used, EDA must be done to check anomalies in the data for cleaning.

In [7]:
try:
    traffic_data = preprocess.read_df_feather(file_dir=DATA_LOCATION,
                                          filename="merged_traffic_data")
    print("Loaded preprocesssed traffic data.")
except:
    # Loading raw data
    traffic_data, traffic_stations = datautils.load_traffic_datasets(
                                                    DATA_LOCATION,
                                                    filenames["TRAFFIC_DATA"],
                                                    filenames["TRAFFIC_STATIONS"])

    print("Processing traffic data.")

    traffic_vol_cols = [col for col in traffic_data.columns 
                    if 'traffic_volume' in col]

    traffic_data = preprocess.process_raw_data(traffic_data,
                                               traffic_stations,
                                               traffic_vol_cols,
                                               fips_loc_df)
    
    # Save processed traffic data for later use
    preprocess.save_df_feather(df=traffic_data,
                            file_dir=DATA_LOCATION,
                            filename="merged_traffic_data",
                            verbose=True)

Loaded preprocesssed traffic data.


In [8]:
fips = 6

In [9]:
fips_state_ref[fips]

'california'

In [10]:
sub_df = traffic_data[traffic_data["fips_state_code"] == 
                      fips].sort_values(by="date").reset_index(drop=True)

In [11]:
sub_df.shape

(191185, 34)

In [12]:
sub_df.groupby(["date", "station_id"]).head()

,date,traffic_volume_counted_after_0000_to_0100,traffic_volume_counted_after_0100_to_0200,traffic_volume_counted_after_0200_to_0300,traffic_volume_counted_after_0300_to_0400,traffic_volume_counted_after_0400_to_0500,traffic_volume_counted_after_0500_to_0600,traffic_volume_counted_after_0600_to_0700,traffic_volume_counted_after_0700_to_0800,traffic_volume_counted_after_0800_to_0900,...,traffic_volume_counted_after_2300_to_2400,station_id,fips_state_code,functional_classification_name,direction_of_travel_name,longitude,fips_county_code,latitude,daily_tot_vol,daily_avg_vol
0,2015-01-01,3838,4378,3177,1979,1547,2052,829,2378,2843,...,2806,074870,6,Urban: Principal Arterial - Other Freeways or ...,North,118.458582,37,34.158876,88307,3679.458333
1,2015-01-01,43,54,57,31,27,38,39,72,100,...,37,035730,6,Rural: Minor Arterial,South,122.014027,21,39.521813,3100,129.166667
2,2015-01-01,64,54,31,24,23,46,48,64,109,...,58,035710,6,Rural: Principal Arterial - Other,North,121.688095,7,39.415027,3344,139.333333
3,2015-01-01,462,365,258,176,159,228,369,542,723,...,345,023040,6,Urban: Principal Arterial - Interstate,North,122.360431,89,40.570947,21680,903.333333
4,2015-01-01,60,32,21,9,13,14,26,35,91,...,34,032080,6,Urban: Principal Arterial - Other,South,121.065652,61,38.904086,3095,128.958333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191180,2015-12-31,155,103,94,82,193,551,651,652,598,...,279,062320,6,Urban: Principal Arterial - Other,West,119.693011,31,36.314007,17001,708.375000
191181,2015-12-31,25,20,4,2,3,4,13,29,147,...,17,099060,6,Rural: Principal Arterial - Other,South,118.477548,27,37.380907,2281,95.041667
191182,2015-12-31,623,389,251,186,328,706,1570,2608,3249,...,1300,055490,6,Urban: Principal Arterial - Other Freeways or ...,North,119.727979,83,34.426955,63699,2654.125000
191183,2015-12-31,696,566,431,414,660,1099,1535,1935,2129,...,1133,030150,6,Urban: Principal Arterial - Interstate,South,121.516545,67,38.495536,58870,2452.916667


## 2.4 Functions for extracting features

Other functions are located in *utils/preprocess.py*. 

In [13]:
def get_filtered_df(fips_state_code, save_dir,
                    traffic_data=None, traffic_stations=None, overwrite=False):
    file_path = f"{os.path.join(save_dir, str(fips_state_code))}.pkl"

    if overwrite == False and os.path.isfile(file_path):
        print(f"File already exists. Retrieving DataFrame from '{file_path}'.")
        df = read_df_feather(save_dir, filename=fips_state_code)
        return df

    sub_df_cols = temporal_cols + new_traffic_vol_cols + common_cols
    sub_df = traffic_data[traffic_data["fips_state_code"]
                          == fips_state_code][sub_df_cols]

    df = pd.merge(sub_df, traffic_stations[traffic_stations["fips_state_code"] == fips_state_code]
                  [common_cols + spatial_cols], on=common_cols)
    df["day_vol"] = df[new_traffic_vol_cols].sum(axis=1).values

    save_df_feather(df, save_dir, filename=fips_state_code)

    return df


def get_transformed_vol_df(station_id, sub_df=None, save_dir=None, verbose=True, overwrite=False):
    file_path = f"{os.path.join(save_dir, station_id)}.fea"

    if overwrite == False and os.path.isfile(file_path):
        if verbose:
            print(
                f"File already exists. Retrieving DataFrame from '{file_path}'.")
        processed_df = read_df_feather(save_dir, filename=station_id)
        return processed_df

    col_timestamp = "date"
    col_trafficvol = "traffic_volume"

    station_df = sub_df[sub_df["station_id"]
                        == station_id][historical_vol_cols]
    sum_station_df = pd.DataFrame()
    dates = list(station_df[col_timestamp].unique())

    if verbose:
        print(
            f"Calculating total hourly volume collected per timestamp from station {station_id}.")
        iter = tqdm(dates)
    else:
        iter = dates
    for date in iter:
        date_condition = station_df[col_timestamp] == date
        sum_df = station_df[date_condition].sum()
        sum_df[col_timestamp] = date

        sum_station_df = sum_station_df.append(sum_df, ignore_index=True)

    row_idxs = range(0, sum_station_df.shape[0])
    if verbose:
        print(f"Transforming DataFrame with hourly volume rows.")
        iter = tqdm(row_idxs)
    else:
        iter = row_idxs
    dates = sum_station_df[col_timestamp].to_list()
    hourly_volumes = sum_station_df[new_traffic_vol_cols].to_numpy()

    all_volumes = []
    timestamps = []

    hour_delta = [np.timedelta64(hour, 'h') for hour in range(0, 24)]

    for row_cnt in iter:
        sub_timestamps = [dates[row_cnt] + hour for hour in hour_delta]
        sub_vols = list(hourly_volumes[row_cnt])

        timestamps += sub_timestamps
        all_volumes += sub_vols

    processed_df = pd.DataFrame()
    processed_df[col_timestamp] = timestamps
    processed_df[col_trafficvol] = all_volumes
    processed_df = processed_df.sort_values(
        by=[col_timestamp]).reset_index(drop=True)

    if save_dir != None:
        save_df_feather(df=processed_df,
                        file_dir=save_dir,
                        filename=station_id,
                        verbose=verbose)

    return processed_df


def get_dataset_splits(df, test_count=61, datetime_unit="D", ratio_split=False, temporal_split=True):
    """
    test_count : 61 days for November (30 days) and December (31 days)
    datetime_unit : "D" to indicate days
    """

    col_timestamp = "date"

    val_ratio = .15
    test_ratio = .15
    train_ratio = 1 - (test_ratio + val_ratio)

    temporal_limit = df[col_timestamp].max(
    ) - np.timedelta64(test_count, datetime_unit)

    if temporal_split:
        train_df = df[df[col_timestamp] <= temporal_limit]
        val_df = None
        test_df = df[df[col_timestamp] > temporal_limit]
    elif ratio_split:
        train_range = int(df.shape[0]*train_ratio)
        val_range = int(df.shape[0]*val_ratio)

        train_df = df.iloc[0:train_range]
        val_df = df.iloc[train_range:train_range+val_range]
        test_df = df.iloc[train_range+val_range:]

    return train_df, val_df, test_df


# Based on:
# - https://towardsdatascience.com/fast-and-robust-sliding-window-vectorization-with-numpy-3ad950ed62f5
# - https://machinelearningmastery.com/xgboost-for-time-series-forecasting/

def get_sliding_windows(array, max_time, sub_window_size, stride_size):
    sub_windows = (
        np.expand_dims(np.arange(sub_window_size), 0) +
        np.expand_dims(np.arange(max_time + 1, step=stride_size), 0).T
    )

    array = array[sub_windows]
    X_values = array[:-1, :]
    
    # Assumes first column is for traffic volumes
    y_values = array[1:, -stride_size:,0]

    return X_values, y_values

In [14]:
# Modified from:
# https://stackoverflow.com/questions/22951956/most-efficient-way-to-create-an-array-of-cos-and-sin-in-numpy
def get_sincos(arr):
    theta = 2 * np.pi * arr
    return np.vstack((np.sin(theta),
                      np.cos(theta))).T

In [15]:
# Modified from:
# - https://medium.com/@dan.allison/how-to-encode-the-cyclic-properties-of-time-with-python-6f4971d245c0
# - https://www.kaggle.com/avanwyk/encoding-cyclical-features-for-deep-learning

days_in_month = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

def sin_cos(n):
    theta = 2 * pi * n
    return (sin(theta), cos(theta))


def get_cycles(df):
    '''
    Get the cyclic properties of a datetime,
    represented as points on the unit circle.
    '''

    months = df["date"].dt.month.to_numpy() - 1
    days = df["date"].dt.day.to_numpy() - 1
    weekdays = df["date"].dt.weekday.to_numpy()
    hours = df["date"].dt.hour.to_numpy()
    
    days_in_months = np.array([days_in_month[month] for month in months])
    
    month_cyc = get_sincos(months/12).T
    day_cyc = get_sincos(days/days_in_months).T
    weekday_cyc = get_sincos(weekdays / 7).T
    hour_cyc = get_sincos(months/24).T

    df["month_sin"] = month_cyc[0]
    df["month_cos"] = month_cyc[1]
    
    df["day_sin"] = day_cyc[0]
    df["day_cos"] = day_cyc[1]
    
    df["weekday_sin"] = weekday_cyc[0]
    df["weekday_cos"] = weekday_cyc[1]
    
    df["hour_sin"] = hour_cyc[0]
    df["hour_cos"] = hour_cyc[1]
    
    return df

In [16]:
sub_df["date"]

0        2015-01-01
1        2015-01-01
2        2015-01-01
3        2015-01-01
4        2015-01-01
            ...    
191180   2015-12-31
191181   2015-12-31
191182   2015-12-31
191183   2015-12-31
191184   2015-12-31
Name: date, Length: 191185, dtype: datetime64[ns]

In [17]:
def get_other_features(df, traffic_stations, station_id, fips_state_code):
    cnt = df.shape[0]

    # Temporal
    df = get_cycles(df)
    df["hour_of_day"] = df["date"].dt.hour
    df.loc[(6 <= df["hour_of_day"]) & (df["hour_of_day"] < 12), "part_of_day"] = 1
    df.loc[(12 <= df["hour_of_day"]) & (df["hour_of_day"] < 18), "part_of_day"] = 2
    df.loc[(18 <= df["hour_of_day"]), "part_of_day"] = 3
    df["day_of_week"] = df["date"].dt.dayofweek
    df.loc[df["day_of_week"] <= 4, "is_weekday"] = 1 # True
    df.loc[df["day_of_week"] >= 5, "is_weekday"] = 0 # False
    
    # Can be added for DFs with hourly entries
    df["hour_of_day"] = df["date"].dt.hour
    df["part_of_day"] = [0]*cnt
    
    sub_info = traffic_stations[(traffic_stations["station_id"] == station_id) & 
                     (traffic_stations["fips_state_code"] == fips_state_code)]
    county_code = sub_info["fips_county_code"].unique()[0]
    
    # Provides a combined string for encoding later
    df["loc_info"] = [f"{station_id}-{fips_state_code}-county_code"]*cnt

    return df

In [18]:
# Saves processed data under the data directory

PROC_LOCATION = os.path.join(DATA_LOCATION, 'processed')
datautils.create_folder(PROC_LOCATION)

Folder 'C:\Users\combi\Documents\gitrepos\us-traffic\src\data\processed' exists.


In [19]:
# For this sample, 7 days prior are used as data input and
# 1 day is forecasted (24 hrs)

# In hours
input_window_size = 24*7
output_window_size = 24

In [20]:
feature_cols = ['traffic_volume',
                 'month_sin',
                 'month_cos',
                 'day_sin',
                 'day_cos',
                 'weekday_sin',
                 'weekday_cos',
                 'hour_sin',
                 'hour_cos',
                 'part_of_day',
                 'loc_info']

In [21]:
get_other_features(sub_df, sub_df, "074870", fips)

,date,traffic_volume_counted_after_0000_to_0100,traffic_volume_counted_after_0100_to_0200,traffic_volume_counted_after_0200_to_0300,traffic_volume_counted_after_0300_to_0400,traffic_volume_counted_after_0400_to_0500,traffic_volume_counted_after_0500_to_0600,traffic_volume_counted_after_0600_to_0700,traffic_volume_counted_after_0700_to_0800,traffic_volume_counted_after_0800_to_0900,...,day_cos,weekday_sin,weekday_cos,hour_sin,hour_cos,hour_of_day,part_of_day,day_of_week,is_weekday,loc_info
0,2015-01-01,3838,4378,3177,1979,1547,2052,829,2378,2843,...,1.00000,0.433884,-0.900969,0.000000,1.000000,0,0,3,1.0,074870-6-county_code
1,2015-01-01,43,54,57,31,27,38,39,72,100,...,1.00000,0.433884,-0.900969,0.000000,1.000000,0,0,3,1.0,074870-6-county_code
2,2015-01-01,64,54,31,24,23,46,48,64,109,...,1.00000,0.433884,-0.900969,0.000000,1.000000,0,0,3,1.0,074870-6-county_code
3,2015-01-01,462,365,258,176,159,228,369,542,723,...,1.00000,0.433884,-0.900969,0.000000,1.000000,0,0,3,1.0,074870-6-county_code
4,2015-01-01,60,32,21,9,13,14,26,35,91,...,1.00000,0.433884,-0.900969,0.000000,1.000000,0,0,3,1.0,074870-6-county_code
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191180,2015-12-31,155,103,94,82,193,551,651,652,598,...,0.97953,0.433884,-0.900969,0.258819,-0.965926,0,0,3,1.0,074870-6-county_code
191181,2015-12-31,25,20,4,2,3,4,13,29,147,...,0.97953,0.433884,-0.900969,0.258819,-0.965926,0,0,3,1.0,074870-6-county_code
191182,2015-12-31,623,389,251,186,328,706,1570,2608,3249,...,0.97953,0.433884,-0.900969,0.258819,-0.965926,0,0,3,1.0,074870-6-county_code
191183,2015-12-31,696,566,431,414,660,1099,1535,1935,2129,...,0.97953,0.433884,-0.900969,0.258819,-0.965926,0,0,3,1.0,074870-6-county_code


## 2.5 Guide for using pre-processed data as input (LSTM)

For this section, the guide shows a sample for using the data as input to an LSTM model. The code blocks are commented out as they require pre-processed data which is provided as an output by *preprocess_trafficdata.py*.

In [22]:
# Instead of setting it as 24 immediately, we can prepare the model to prepare
# predictions and take as input the forecasted value from the previous timestamps

# In hours
input_window_size = 24*7
output_window_size = 24

# Selected sample features
feature_cols = ['traffic_volume',
                 'month_sin',
                 'month_cos',
                 'day_sin',
                 'day_cos',
                 'weekday_sin',
                 'weekday_cos',
                 'hour_sin',
                 'hour_cos',
                 'part_of_day',
                 'loc_info']

In [23]:
# Requires further optimization

# MODEL_LOCATION = os.path.join(os.getcwd(), 'models')
# create_folder(MODEL_LOCATION)

# codes = [6] #12, 29, 28, 51, 39, 13, 55, 53, 16, 36, 40, 28, 29]
# verbose = False

# X_train = np.empty((0, input_window_size, len(feature_cols)))
# y_train = np.empty((0, output_window_size))

# X_test = np.empty((0, input_window_size, len(feature_cols)))
# y_test = np.empty((0, output_window_size))

# for fips_state_code in tqdm(codes):
#     model_input_dir = os.path.join(processed_dir, str(fips_state_code))
#     create_folder(model_input_dir)

#     station_ids = [x.split('.')[0] for x in os.listdir(model_input_dir)]

#     for station_id in tqdm(station_ids):
#         processed_df = get_transformed_vol_df(station_id,
#                                                           save_dir=model_input_dir,
#                                                           verbose=verbose)
#         train_df, _, test_df = get_dataset_splits(processed_df)

#         train_df = get_other_features(train_df, 
#                                        traffic_stations, 
#                                        station_id, 
#                                        fips_state_code)

#         arr = train_df[feature_cols].to_numpy()
#         sub_X_train, sub_y_train = get_sliding_windows(array=arr,
#                                                       max_time=arr.shape[0]-input_window_size,
#                                                       sub_window_size=input_window_size,
#                                                       stride_size=output_window_size)

#         X_train = np.append(X_train, sub_X_train, axis=0)
#         y_train = np.append(y_train, sub_y_train, axis=0)

#         test_df = get_other_features(test_df, 
#                                        traffic_stations, 
#                                        station_id, 
#                                        fips_state_code)
#         arr = test_df[feature_cols].to_numpy()
#         sub_X_test, sub_y_test = get_sliding_windows(array=arr,
#                                   max_time=arr.shape[0]-input_window_size,
#                                   sub_window_size=input_window_size,
#                                   stride_size=output_window_size)

#         X_test = np.append(X_test, sub_X_test, axis=0)
#         y_test = np.append(y_test, sub_y_test, axis=0)

A class dataset is also provided with additional functions and configurations. For this case, it takes as input historical traffic volume per hour for a range of 7 days and provides a forecast of 1 day with 24 data points (hourly). It also takes as input the part of day, and combined location features.

In [24]:
class Dataset:
    # Use class instead of multiple returns
    def __init__(self, X_train, y_train, X_test, y_test, locations):
        self.X_train = X_train
        self.y_train = y_train.astype('float32')
        self.X_test = X_test
        self.y_test = y_test.astype('float32')
        self.day_count = 7
        self.hour_interval = 6
        self.hour_ranges = range(0, 24, self.hour_interval)

        self.X_val = None
        self.y_val = None

        # Station IDs
        from sklearn.preprocessing import LabelEncoder
        self.le = LabelEncoder()
        self.le.fit(locations)

        self.input_window_size = 24*7
        self.output_window_size = 24

    def get_split_input(self, X_values):
        # Traffic volume
        traffic_vols = np.asarray(X_values[:,:,0]).astype('float32')
        traffic_vols = traffic_vols.reshape(-1, self.input_window_size, 1)

        # Temporal features (cyclical timestamps)
        cyclical_timestamps = X_values[:,:,1:-2].reshape(-1, self.input_window_size*8, 1)
        cyclical_timestamps = np.asarray(cyclical_timestamps).astype('float32') 

        # Part of day
        part_of_day = X_values[:,:,-2:-1].astype(int)

        # Location feature
        encoded_locs = self.le.transform(X_values[:,:,-1:].reshape(-1))
        encoded_locs = encoded_locs.reshape(-1, self.input_window_size, 1).astype(int)

        return [traffic_vols,
                cyclical_timestamps,
                part_of_day,
                encoded_locs]

    def create_validation_set(self, val_ratio=0.10):
        # Retrieve random indices for validation and train
        train_len = self.X_train.shape[0]
        val_len = int(train_len*val_ratio)
        val_idxs = random.sample(range(0, train_len), val_len)
        train_idxs = list(set(range(0, train_len)) - set(val_idxs))

        # Retrieve new length
        train_len = len(train_idxs)

        # Modify val before original
        self.X_val = self.X_train[val_idxs]
        self.X_train = self.X_train[train_idxs]

        self.y_val = self.y_train[val_idxs]
        self.y_train = self.y_train[train_idxs]


    def get_reshaped_splits(self, get_val=True):
        if get_val == True:
            create_validation_set()
            # Reshape validation input
            self.X_val = self.X_val.reshape(val_len,-1,self.day_count)
        train_len = self.X_train.shape[0]
        test_len = self.X_test.shape[0]
        # Reshape test and train input
        self.X_train = self.X_train.reshape(train_len,-1,self.day_count)
        self.X_test = self.X_test.reshape(test_len,-1,self.day_count)


    def get_sub_y(self, y_values, part_of_day):
        # [0:6) - midnight to morning
        # [6:12) - morning to afternoon
        # [12:18) - afternoon to night
        # [18:23] - night to midnight

        hour_idx = self.hour_ranges[part_of_day]
        y_sub = y_values[:, hour_idx:hour_idx+self.hour_interval]
        
        return y_sub

In [25]:
# all_locs = np.unique(X_train[:,:,-1:])

# data = Dataset(X_train, y_train,
#                X_test, y_test,
#                all_locs)
# data.create_validation_set()
# X_train_split = data.get_split_input(data.X_train)

In [26]:
# from tensorflow.keras.models import Model, Sequential
# from tensorflow.keras.layers import Input, Embedding, Flatten, BatchNormalization, \
#                                     Concatenate, Reshape, Dense, LSTM, Dropout
# from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
# from tensorflow.keras.optimizers import RMSprop
# import tensorflow.keras.backend as K
# import tensorflow as tf

# def rmse(y_true, y_pred):
#     return K.sqrt(K.mean(K.square(y_pred - y_true)))

Below is a sample LSTM model used to forecast 24 hours of traffic volume for a given day. Still a work in a progress as the RMSE values for this model is still relatively high.

In [27]:
# import tensorflow.keras.models as M

# def get_lstm_model(model_input, all_locs, checkpt_path):

#     SINCOS_FEATS = 8 # Month, Day, Weekday, Hour
#     VOL_FEATS = 1
#     PARTS_OF_DAY = 4
#     LOOKBACK_SIZE = 24*7
#     FORECAST_PERIOD = 24 # in hours
#     LOC_CNT = len(all_locs)

#     input_dim = LOOKBACK_SIZE

#     # Traffic Volume
#     input_vol_layer = Input(input_dim, )
#     x1 = BatchNormalization()(input_vol_layer)
#     x1 = Dense(VOL_FEATS * 128, activation='selu')(x1)

#     # SinCos Temporal Features
#     input_time_layer = Input(shape=(input_dim*SINCOS_FEATS,))
#     x2 = BatchNormalization()(input_time_layer, )
#     x2 = Dense(SINCOS_FEATS*64, activation='selu')(x2)

#     # Part of Day
#     input_pod_layer = Input(shape=(input_dim,))
#     x3 = Embedding(PARTS_OF_DAY, 2)(input_pod_layer)
#     x3 = Flatten()(x3)

#     # Station ID
#     input_location_layer = Input(shape=(input_dim,))
#     x4 = Embedding(LOC_CNT, 2)(input_location_layer)
#     x4 = Flatten()(x4)

#     # main stream
#     x = Concatenate(axis=1)([x1, x2, x3, x4])

#     x = BatchNormalization()(x)
#     x = Dropout(0.3)(x)
#     x = Dense(1024, activation='selu')(x)

#     x = BatchNormalization()(x)
#     x = Dropout(0.3)(x)
#     x = Dense(512, activation='selu')(x)

#     x = Reshape((1, -1))(x)
#     x = BatchNormalization()(x)
#     x = LSTM(512, dropout=0.4, recurrent_dropout=0.2, 
#              return_sequences=True, activation='selu')(x)
#     x = LSTM(256, dropout=0.2, recurrent_dropout=0.2, 
#     return_sequences=True, activation='selu')(x)
#     x = LSTM(128, dropout=0.1, return_sequences=False, 
#              activation='selu')(x)

#     output_layer = Dense(FORECAST_PERIOD, name="trafficvolume")(x)

#     model = M.Model([input_vol_layer,
#                      input_time_layer,
#                      input_pod_layer,
#                      input_location_layer], 
#                     [output_layer])


#     rmsprop = RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
#     model.compile(loss=rmse, optimizer=rmsprop, metrics=[rmse])

#     callbacks = [ReduceLROnPlateau(monitor='val_loss', 
#                                    factor=0.1, patience=3, 
#                                    verbose=1, min_delta=1e-4, mode='min'),
#                  ModelCheckpoint(checkpt_path, 
#                                  monitor='val_loss', verbose=0, 
#                                  save_best_only=True, save_weights_only=True, mode='min'),
#                  EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=5, mode='min',
#                                baseline=None, restore_best_weights=True)
#             ]

#     return model, callbacks